In [1]:
# Dependencies
import pandas as pd
import requests
import psycopg2
from sqlalchemy import create_engine, table

In [2]:
# Importing recipes from spoonacular
api_key = "a21364208c9d43719315d65c34faa384"  ### will need to grab from config file
# Using url for "Get Recipe Information" JSONs: https://spoonacular.com/food-api/docs#Get-Recipe-Information
# url = "https://api.spoonacular.com/recipes/10/information?includeNutrition=false"

id_df = pd.read_csv('unique_recipes.csv')
id = id_df['0']

# id = range(10)

# Initializing lists for url and query_url
url = []
query_url = [None]*len(id)

# Loop through a set of recipes up to a pre-determined amount
for i in range(0, len(id)):
    url.append(f"https://api.spoonacular.com/recipes/{id[i]}/information?includeNutrition=false")
    query_url[i] = f'{url[i]}&apiKey={api_key}'


In [3]:
from pprint import pprint
# Initializing lists for results from query
recipe_id = []
recipe_name = []
servings = []
diets = []
dairyFree = []
vegan = []
vegetarian = []
ketogenic = []
images = []
source_url = []
ingredients = []
individual_ingre=[]

# Looping through recipe JSONs
count = 0
ingred_list = []

### Change the range later after number of recipes is agreed on.
for i in range(0, len(id)):
    
    try:
        random_list=[]
        response = requests.get(query_url[i]).json()
    
        recipe_id.append(response['id'])
        recipe_name.append(response['title'])
        servings.append(response['servings'])
        diets.append(response['diets'])
        dairyFree.append(response['dairyFree'])
        vegan.append(response['vegan'])
        vegetarian.append(response['vegetarian'])
#         ketogenic.append(response['ketogenic'])
        images.append(response['image'])
        source_url.append(response['sourceUrl'])
        ingredients.append(response['extendedIngredients'])
        
        for ingre in ingredients:
            for i in range(len(ingre)):
                random_list.append(ingre[i]["name"])
        individual_ingre.append(random_list)
    except:
        print(f'Recipe {id[i]} was not found.')

Recipe 663641 was not found.
Recipe 664025 was not found.
Recipe 655847 was not found.
Recipe 639606 was not found.
Recipe 664780 was not found.
Recipe 640259 was not found.
Recipe 648524 was not found.
Recipe 641202 was not found.
Recipe 641671 was not found.
Recipe 633537 was not found.
Recipe 641908 was not found.
Recipe 634660 was not found.
Recipe 634751 was not found.
Recipe 659638 was not found.
Recipe 660286 was not found.
Recipe 635712 was not found.
Recipe 660406 was not found.
Recipe 636581 was not found.
Recipe 661291 was not found.
Recipe 637222 was not found.
Recipe 653775 was not found.
Recipe 646491 was not found.
Recipe 663121 was not found.
Recipe 663227 was not found.


In [14]:
# Develop dataframe from queried recipes
recipe_df = pd.DataFrame({'ID': recipe_id,
                          'Recipe Name': recipe_name,
                          'Servings': servings,
                          'Diets': diets,
                          'Dairy Free': dairyFree,
                          'Vegan': vegan,
                          'Vegetarian': vegetarian
## It looks like some recipes don't have an Image URL, source URL, or Ingredients list.
## So these lists come up short compared to the above lists. 
## Commented out for the time being. Might need to add an "if" statement during the appending in the above cell.
#                           'Image URL': images, 
#                           'Source URL': source_url,
#                           'Ingredients': individual_ingre
                        })
                        
recipe_forsql_df = recipe_df.rename(columns = {'Recipe Name':'Recipe_Name',
                                               'Dairy Free':'Dairy_Free',
                                               'Image URL':'Image_URL',
                                               'Source URL':'Source_URL'
                                              })
# Display recipe_df
recipe_df.head()

,ID,Recipe Name,Servings,Diets,Dairy Free,Vegan,Vegetarian
0,1515523,Instant Pot BBQ Chicken Thighs,4,"[gluten free, dairy free]",True,False,False
1,663559,Tomato and lentil soup,4,"[gluten free, dairy free, lacto ovo vegetarian...",True,True,True
2,638996,Chocolate Dipped Coconut Macaroons,9,[gluten free],False,False,False
3,647197,Honey Chipotle Pork Ribs,16,"[gluten free, dairy free]",True,False,False
4,655392,Pear and Apple Crumble,8,"[dairy free, lacto ovo vegetarian, vegan]",True,True,True


In [15]:
rds_connection_string = 'postgres:postgres@localhost:5432/Project_2'
engine = create_engine(f'postgresql://{rds_connection_string}')

recipe_df.to_sql('recipe_df', con=engine, index=False, if_exists='replace')

In [13]:
len(vegetarian)

1955

In [6]:
len(recipe_name)

1955

In [9]:
len(recipe_id)

1955

In [10]:
len(servings)

1955